In [1]:
pip install jupyter-dash

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports
import dash
from dash import Dash, html, dcc
import pandas as pd
from dash.dependencies import Input, Output, State
#import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from dash import no_update

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df['Outcome'] = np.where(spacex_df['class'] == 1, 'Success', 'Failure')

In [4]:
spacex_df.sample(n=10)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category,Outcome
17,17,19,CCAFS LC-40,1,1952.0,F9 v1.1 B1018,v1.1,Success
49,49,45,CCAFS SLC-40,1,2205.0,F9 FT B1035.2,FT,Success
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0,Failure
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0,Failure
7,7,9,CCAFS LC-40,0,2296.0,F9 v1.1,v1.1,Failure
43,43,38,KSC LC-39A,0,6761.0,F9 FT B1037,FT,Failure
22,22,25,CCAFS LC-40,1,3100.0,F9 FT B1023.1,FT,Success
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0,Failure
53,53,52,CCAFS SLC-40,0,2647.0,F9 B4 B1039.2,B4,Failure
29,29,37,VAFB SLC-4E,1,9600.0,F9 FT B1036.1,FT,Success


In [10]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div(dcc.Dropdown(id='site-dropdown', 
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},                                            
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                                ],     
                                        value='ALL',
                                        placeholder='Select a launch site',
                                        searchable = True,
                                        style={'width': '50%', 'font-size': '20px','padding': '3px'})),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 1000: '1000',2000: '2000',3000: '3000',
                                                      4000: '4000',5000: '5000',6000: '6000',7000: '7000',
                                                      8000: '8000',9000: '9000',10000: '10000',},
                                                value=[min_payload, max_payload])),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site',
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        site_specific_data = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(site_specific_data, 
        names='Outcome', 
        title=('Total Success Launches For Site ' + entered_site))
        return fig
        

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_plot(entered_site, payload):
    payload_subset = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
    if entered_site != 'ALL':
        payload_subset = payload_subset[payload_subset['Launch Site'] == entered_site]
    fig = px.scatter(payload_subset, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jan/2023 23:31:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:31:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:32:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 23:32:28] "POST /_dash-